In [3]:
import numpy as np
import matplotlib.pyplot as plt
from math import log

计算复杂度不高，结果易于理解，对中间值的确实不敏感，可以处理不想干的特征。
缺点：overfitting

$$ Ent(D) =-\sum_{k=1}^{|y|}p_klog_{2}{p_k} $$

In [30]:
def calEnt(dataSet):
    '''
    根据标签计算信息熵
    dataset最后一列是类别标签
    '''
    counts = len(dataSet)
    labels={}
    for v in dataSet:
        className=v[-1]
        if className not in labels:
            labels[className]=0
        labels[className]+=1
        
    ent = 0
    for k in labels.keys():
        p= labels[k]*1.0/counts
        ent -= p*log(p,2)
        
    return ent

def createDataSet():
    dataSet = [[1, 1, 'yes'],
               [1, 1, 'yes'],
               [1, 0, 'no'],
               [0, 1, 'no'],
               [0, 1, 'no']]
    labels = ['no surfacing','flippers']
    #change to discrete values
    return dataSet, labels

In [31]:
dataSet,labels=createDataSet()
calEnt(dataSet)

0.9709505944546686

In [33]:
# 根据指定的列索引和值，过滤数据
def getDs(dataSet, idx, value):
    re=[]
    for v in dataSet:
        if v[idx] == value:
            ## 去除当前属性
            tmp = v[:idx]
            tmp.extend(v[idx+1:])
            re.append(tmp)
    return re
getDs(dataSet,0,1)

[[1, 'yes'], [1, 'yes'], [0, 'no']]

$$Gain(D,a) = Ent(D) - \sum_{v=1}^{V}\frac{|D^v|}{|D|}Ent({D^v})$$

In [34]:
def chooseBestFeatureToSplit(dataSet):
    '''
    选择最佳分类值
    '''
    #获取属性数目
    counts = len(dataSet[0])-1
    ent=calEnt(dataSet)
    
    #针对每个属性计算gain
    gain=-10000
    bastFeature=-1
    
    #遍历各个属性
    for i in range(counts):
        # 获取属性值
        fset=set([v[i]  for v in dataSet])
        newEnt=0
        
        #计算每个属性值条件下的信息熵
        for values in fset:
            #过滤出指定的集合
            ds=splitDs(dataSet, i, values)
            newEnt -= len(ds)*1.0*calShannonEnt(ds)/counts
        
        if newEnt > gain:
            gain = newEnt
            bastFeature=i
    
    return ent+gain, bastFeature
        
chooseBestFeatureToSplit(dataSet)

(-0.40649315662706575, 0)

In [35]:
import operator
def getFrequentClass(classList):
    '''
    获取classList中的占比最多的类别
    '''
    classCount={}
    for vote in classList:
        if vote not in classCount: classCount[vote]=0
        classCount[vote] += 1
    sortCount = sorted(classCount.iterms(), key=operator.itemgetter(1),reverse=True)
    
    return sortCount[0][0]

In [36]:
# m*(属性数量 + 值列) 
def createTree(dataSet,labels):
    ##完全相同则终止
    classList = [ v[-1] for v in dataSet]
    if classList.count(classList[0]) == len(classList):
        return classList[0]
    
    ## 没有属性了则终止，返回最多的那个属性
    if len(dataSet[0]) == 1 :
        return getFrequentClass(classList)
    
    #找到最好的feature，然后按照feature的值
    gain, bestFea=chooseBestFeatureToSplit(dataSet)
    bestFeatureName=labels[bestFea]
    
    vs =[v[bestFea] for v in dataSet]
    vs = set(vs)
    myTree = {bestFeatureName:{}}
    del(labels[bestFea])
    #开始切分
    for featureValue in vs:
        sublabels=labels[:]
        myTree[bestFeatureName][featureValue] = createTree( \
            splitDs(dataSet, bestFea, featureValue), sublabels)
        
    return myTree

In [37]:
createTree(dataSet,labels.copy())

{'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}

In [38]:
def classify2(inputTree,fLables,vec):
    '''
    开始分类
    inputTree 分类树
    fLabels 名称
    vec
    '''
    print("=>",inputTree, featLabels)
    if isinstance(inputTree,str): return inputTree
    labelNow = list(inputTree.keys())[0]
    labelIdx = -1
    for i,v in enumerate(fLables):
        if v == labelNow: labelIdx =i
    
    subs = inputTree[labelNow]
    vecFeature=vec[labelIdx]
    
    for v in subs:
        if v == vecFeature:
            return classify2(subs[v],fLables,vec)
    

In [39]:
tree={'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}}
print(labels)
classify(tree,labels,[1,1])

['no surfacing', 'flippers']
=> {'no surfacing': {0: 'no', 1: {'flippers': {0: 'no', 1: 'yes'}}}} ['no surfacing', 'flippers']
=> {'flippers': {0: 'no', 1: 'yes'}} ['no surfacing', 'flippers']
=> yes ['no surfacing', 'flippers']


'yes'

In [21]:
def storeTree(inputTree,filename):
    import pickle
    fw=open(filename,'w')
    pickle.dump(inputTree,fw)
    fw.close()

------------新的例子

In [196]:
## 眼睛类型预测
fw=open('./datas/lenses.txt')
datas = [line.strip().split('\t') for line in fw.readlines()]
lensesLabels=['age','prescript','astigmatic','tearRate','result']
rs=createTree(datas,lensesLabels)
print(rs)

{'tearRate': {'reduced': 'no lenses', 'normal': {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}}, 'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'presbyopic': 'no lenses', 'young': 'hard', 'pre': 'no lenses'}}}}}}}}


In [198]:
print( classify(rs,lensesLabels,['young','hyper','no','normal']))

=================== {'tearRate': {'reduced': 'no lenses', 'normal': {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}}, 'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'presbyopic': 'no lenses', 'young': 'hard', 'pre': 'no lenses'}}}}}}}} ['age', 'prescript', 'astigmatic', 'result']
=================== {'astigmatic': {'no': {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}}, 'yes': {'prescript': {'myope': 'hard', 'hyper': {'age': {'presbyopic': 'no lenses', 'young': 'hard', 'pre': 'no lenses'}}}}}} ['age', 'prescript', 'astigmatic', 'result']
=================== {'age': {'presbyopic': {'prescript': {'myope': 'no lenses', 'hyper': 'soft'}}, 'young': 'soft', 'pre': 'soft'}} ['age', 'prescript', 'astigmatic', 'result']
=================== soft ['age', 'prescript', 'astigmatic', 'result']
soft
